In [3]:
import pandas as pd
import numpy as np
from datetime import date
from QuantLib import *

# Step 1: Collect option prices (example dataset)
options_data = pd.read_csv('options_data.csv')

# Step 2: Define the pricing engine and model
today = date.today()
calendar = TARGET()
day_count = Actual365Fixed()

# Define the underlying asset
spot_price = 100.0
risk_free_rate = 0.05
dividend_yield = 0.0

underlying = SimpleQuote(spot_price)
risk_free_curve = FlatForward(today, risk_free_rate, day_count)
dividend_curve = FlatForward(today, dividend_yield, day_count)

# Step 3: Calculate implied volatilities
implied_vols = []
for index, row in options_data.iterrows():
    option_type = Option.Call if row['OptionType'] == 'Call' else Option.Put
    expiration_date = date(int(row['Year']), int(row['Month']), int(row['Day']))
    strike_price = row['Strike']
    option_price = row['Price']
    
    # Construct the option
    payoff = PlainVanillaPayoff(option_type, strike_price)
    exercise = EuropeanExercise(expiration_date)
    european_option = EuropeanOption(payoff, exercise)

    # Define the option pricing engine
    process = BlackScholesMertonProcess(
        QuoteHandle(underlying),
        YieldTermStructureHandle(dividend_curve),
        YieldTermStructureHandle(risk_free_curve),
        BlackVolTermStructureHandle(FlatVol(today, 0.0, day_count))
    )
    european_option.setPricingEngine(AnalyticEuropeanEngine(process))

    # Calculate the implied volatility
    implied_vol = european_option.impliedVolatility(option_price, process)
    implied_vols.append(implied_vol)

# Step 4: Create a grid
strike_prices = options_data['Strike'].unique()
maturities = options_data['Expiration'].unique()

# Step 5: Interpolate and smooth (omitted for brevity)
# Convert strike prices and maturities to numeric format
numeric_strikes = np.array(strike_prices, dtype=float)
numeric_maturities = np.array([(expiration - today).days / 365.0 for expiration in maturities], dtype=float)

# Create a meshgrid
X, Y = np.meshgrid(numeric_maturities, numeric_strikes)
Z = np.array(implied_vols).reshape(X.shape)

# Perform interpolation and smoothing (example using scipy.interpolate)
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter

# Step 6: Visualize the surface (omitted for brevity)
# Interpolate the volatility surface
interpolated_surface = griddata((numeric_maturities, numeric_strikes), implied_vols, (X, Y), method='linear')

# Smooth the surface using a Gaussian filter
smoothed_surface = gaussian_filter(interpolated_surface, sigma=1.0)

# Step 6: Visualize the surface
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, smoothed_surface, cmap='viridis')
ax.set_xlabel('Maturity')
ax.set_ylabel('Strike Price')
ax.set_zlabel('Implied Volatility')
ax.set_title('Volatility Surface')
plt.show()

array([4, 6])

In [5]:
np.sum([[1,2],[3,4]],axis=1)

array([3, 7])